In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, GlueDataset, default_data_collator
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import mlflow
import matplotlib.pyplot as plt

/Applications/anaconda3/envs/BERT-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("../data/train_df.csv")
data = data.dropna(subset=['words'])
data = data.sample(10000)

In [5]:
from transformers import RobertaTokenizer

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
max_len = 128

In [7]:
def encode_text(texts, tokenizer, max_len):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
                            text,
                            add_special_tokens=True,
                            max_length=max_len,
                            padding='max_length',
                            return_attention_mask=True,
                            return_tensors='pt',
                       )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

In [9]:
input_ids, attention_masks = encode_text(data['text'], tokenizer, max_len)
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, data['target'].values, test_size=0.1, random_state=42)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.1, random_state=42)

In [10]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
eval_dataset = TensorDataset(validation_inputs, validation_masks, validation_labels)

/var/folders/kl/3nc79ycx61v13jbqlw61nnfw0000gn/T/ipykernel_29061/2933568192.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs)
/var/folders/kl/3nc79ycx61v13jbqlw61nnfw0000gn/T/ipykernel_29061/2933568192.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_inputs = torch.tensor(validation_inputs)
/var/folders/kl/3nc79ycx61v13jbqlw61nnfw0000gn/T/ipykernel_29061/2933568192.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_masks = torch.tensor(train_masks)
/var/folders/kl/3nc79ycx61

In [11]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

/Applications/anaconda3/envs/BERT-test/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def my_data_collator(features):
    batch = {}
    batch['input_ids'] = torch.stack([f[0] for f in features])
    batch['attention_mask'] = torch.stack([f[1] for f in features])
    batch['labels'] = torch.tensor([f[2] for f in features])
    return batch

In [13]:
training_args = TrainingArguments(
    output_dir='./results_roberta',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    logging_dir='./logs_roberta',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    warmup_steps=0,
    weight_decay=0.01,
    seed=42,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

In [14]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_predictions):
    labels = eval_predictions.label_ids
    preds = eval_predictions.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    return {
        'eval_accuracy': accuracy,
    }

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=my_data_collator,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

  1%|          | 10/846 [02:50<3:47:52, 16.36s/it]

{'loss': 0.6884, 'grad_norm': 1.1090093851089478, 'learning_rate': 1.9763593380614657e-05, 'epoch': 0.04}


  2%|▏         | 20/846 [05:29<3:42:47, 16.18s/it]

{'loss': 0.6939, 'grad_norm': 0.974604606628418, 'learning_rate': 1.9527186761229316e-05, 'epoch': 0.07}


  4%|▎         | 30/846 [08:03<3:26:20, 15.17s/it]

...

 99%|█████████▉| 840/846 [3:48:04<01:27, 14.63s/it]

{'loss': 0.2581, 'grad_norm': 14.951705932617188, 'learning_rate': 1.4184397163120568e-07, 'epoch': 2.98}


100%|██████████| 846/846 [3:49:23<00:00, 16.27s/it]

{'train_runtime': 13763.1391, 'train_samples_per_second': 1.962, 'train_steps_per_second': 0.061, 'train_loss': 0.3720646234268838, 'epoch': 3.0}


TrainOutput(global_step=846, training_loss=0.3720646234268838, metrics={'train_runtime': 13763.1391, 'train_samples_per_second': 1.962, 'train_steps_per_second': 0.061, 'total_flos': 1775999623680000.0, 'train_loss': 0.3720646234268838, 'epoch': 3.0})

In [17]:
model_path = "./model_roberta"
tokenizer_path = "./tokenizer_roberta"

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

('./tokenizer_roberta/tokenizer_config.json',
 './tokenizer_roberta/special_tokens_map.json',
 './tokenizer_roberta/vocab.json',
 './tokenizer_roberta/merges.txt',
 './tokenizer_roberta/added_tokens.json')

In [18]:
tokenizer_path = "./tokenizer_roberta"
model_path = "./model_roberta"

tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)

In [19]:
from transformers import pipeline

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
text = "Hello, how are you?"

result = classifier(text)
print(result)

[{'label': 'LABEL_1', 'score': 0.9570664763450623}]


In [20]:
text = "I want you to die"

result = classifier(text)
print(result)

[{'label': 'LABEL_0', 'score': 0.9770116209983826}]


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import torch
from transformers import BertTokenizer, BertForSequenceClassification

data = pd.read_csv("../Data/train_df.csv")
data = data.dropna(subset=['words'])
data = data.sample(1000)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("chemin/vers/votre/modele")

# Prétraiter les données de test
test_texts = ["Hello, how are you?"]  # Mettez vos propres données de test ici
test_input_ids = []
test_attention_masks = []
for text in test_texts:
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens=True,
                        max_length=128,
                        padding='max_length',
                        return_attention_mask=True,
                        return_tensors='pt',
                   )
    test_input_ids.append(encoded_dict['input_ids'])
    test_attention_masks.append(encoded_dict['attention_mask'])

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

# Effectuer les prédictions
with torch.no_grad():
    outputs = model(test_input_ids, attention_mask=test_attention_masks)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)

# Calculer les métriques de performance
accuracy = accuracy_score(data['target'], predictions)
precision = precision_score(data['target'], predictions)
recall = recall_score(data['target'], predictions)
f1 = f1_score(data['target'], predictions)
fpr, tpr, thresholds = roc_curve(data['target'], predictions)
roc_auc = auc(fpr, tpr)
conf_matrix = confusion_matrix(data['target'], predictions)

# Enregistrer les résultats dans MLflow
import mlflow

artifact_path = './artifacts/'

with mlflow.start_run(run_name="bert-base-uncased"):
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1_Score", f1)
    mlflow.log_metric("AUC", roc_auc)

    conf_matrix_path = f"{artifact_path}confusion_matrix.csv"
    pd.DataFrame(conf_matrix).to_csv(conf_matrix_path, index=False, header=False)
    mlflow.log_artifact(conf_matrix_path, "metrics")

    plt.figure(figsize=(8, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.grid(True)
    roc_curve_path = f"{artifact_path}roc_curve.png"
    plt.savefig(roc_curve_path)
    plt.close()
    mlflow.log_artifact(roc_curve_path, "plots")
